In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
#loading the datasets
train_data=pd.read_csv("/kaggle/input/titanic/train.csv")
test_data=pd.read_csv("/kaggle/input/titanic/test.csv")


In [3]:
# handling the missing values in training datset
clean=train_data
clean_test=test_data
clean["Embarked"].fillna(clean["Embarked"].mode()[0],inplace=True)
clean_test["Embarked"].fillna(clean_test["Embarked"].mode()[0],inplace=True)

clean["Age"].fillna(clean["Age"].median(),inplace=True)
clean_test["Age"].fillna(clean_test["Age"].median(),inplace=True)



/tmp/ipykernel_13/2122230628.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean["Embarked"].fillna(clean["Embarked"].mode()[0],inplace=True)
/tmp/ipykernel_13/2122230628.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [4]:
#extracting new features from name

clean["Title"]=clean["Name"].str.extract("([A-Za-z]+)\.",expand=False)
clean_test["Title"]=clean_test["Name"].str.extract("([A-Za-z]+)\.",expand=False)

## here nobles titles are replaced with Rare and diffrent forms of words miss and misses are replaced
clean["Title"]=clean["Title"].replace(["Mlle","Ms"],"Miss").replace(["Mme"], "Mrs").replace(
    ["Don","Dona","Rev","Dr","Major","Lady","Sir","Col","Capt","Countess","Jonkheer"], "Rare"
)
clean_test["Title"]=clean_test["Title"].replace(["Mlle","Ms"],"Miss").replace(["Mme"], "Mrs").replace(
    ["Don","Dona","Rev","Dr","Major","Lady","Sir","Col","Capt","Countess","Jonkheer"], "Rare"
)

## extracting deck from cabin 
clean["Deck"]=clean["Cabin"].str[0]
clean["Deck"]=clean["Deck"].fillna("unknown")
clean_test["Deck"]=clean_test["Cabin"].str[0]
clean_test["Deck"]=clean_test["Deck"].fillna("unknown")

## extracting features related to family 
clean["FamilySize"]=clean["SibSp"]+clean["Parch"]+1
clean["Alone"]=(clean["FamilySize"]==1).astype(int)

clean_test["FamilySize"]=clean_test["SibSp"]+clean_test["Parch"]+1
clean_test["Alone"]=(clean_test["FamilySize"]==1).astype(int)

#now categorizing fare and age into diff categories 4 for fare
clean["AgeGroup"]=pd.cut(clean["Age"],bins=[0,12,20,40,60,80],labels=False)
clean["FareGroup"]=pd.cut(clean["Fare"],4,labels=False)

clean_test["AgeGroup"]=pd.cut(clean_test["Age"],bins=[0,12,20,40,60,80],labels=False)
clean_test["FareGroup"]=pd.cut(clean_test["Fare"],4,labels=False)




In [5]:
# now we have extracted features so we will drop unnecessary columns
clean=clean.drop(["Name","Ticket","Cabin"],axis=1)
clean

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,Alone,AgeGroup,FareGroup
0,1,0,3,male,22.0,1,0,7.2500,S,Mr,unknown,2,0,2,0
1,2,1,1,female,38.0,1,0,71.2833,C,Mrs,C,2,0,2,0
2,3,1,3,female,26.0,0,0,7.9250,S,Miss,unknown,1,1,2,0
3,4,1,1,female,35.0,1,0,53.1000,S,Mrs,C,2,0,2,0
4,5,0,3,male,35.0,0,0,8.0500,S,Mr,unknown,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,Rare,unknown,1,1,2,0
887,888,1,1,female,19.0,0,0,30.0000,S,Miss,B,1,1,1,0
888,889,0,3,female,28.0,1,2,23.4500,S,Miss,unknown,4,0,2,0
889,890,1,1,male,26.0,0,0,30.0000,C,Mr,C,1,1,2,0


In [6]:
clean_test=clean_test.drop(["Name","Ticket","Cabin"],axis=1)
clean_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,Alone,AgeGroup,FareGroup
0,892,3,male,34.5,0,0,7.8292,Q,Mr,unknown,1,1,2,0.0
1,893,3,female,47.0,1,0,7.0000,S,Mrs,unknown,2,0,3,0.0
2,894,2,male,62.0,0,0,9.6875,Q,Mr,unknown,1,1,4,0.0
3,895,3,male,27.0,0,0,8.6625,S,Mr,unknown,1,1,2,0.0
4,896,3,female,22.0,1,1,12.2875,S,Mrs,unknown,3,0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,27.0,0,0,8.0500,S,Mr,unknown,1,1,2,0.0
414,1306,1,female,39.0,0,0,108.9000,C,Rare,C,1,1,2,0.0
415,1307,3,male,38.5,0,0,7.2500,S,Mr,unknown,1,1,2,0.0
416,1308,3,male,27.0,0,0,8.0500,S,Mr,unknown,1,1,2,0.0


In [7]:
## now encoding the text into numbers to feed it into classifier
from sklearn.preprocessing import LabelEncoder
for col in ["Sex","Embarked","Title","Deck"]:
    le = LabelEncoder()
    clean[col] = le.fit_transform(clean[col])
clean

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,Alone,AgeGroup,FareGroup
0,1,0,3,1,22.0,1,0,7.2500,2,2,8,2,0,2,0
1,2,1,1,0,38.0,1,0,71.2833,0,3,2,2,0,2,0
2,3,1,3,0,26.0,0,0,7.9250,2,1,8,1,1,2,0
3,4,1,1,0,35.0,1,0,53.1000,2,3,2,2,0,2,0
4,5,0,3,1,35.0,0,0,8.0500,2,2,8,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,13.0000,2,4,8,1,1,2,0
887,888,1,1,0,19.0,0,0,30.0000,2,1,1,1,1,1,0
888,889,0,3,0,28.0,1,2,23.4500,2,1,8,4,0,2,0
889,890,1,1,1,26.0,0,0,30.0000,0,2,2,1,1,2,0


In [8]:
from sklearn.preprocessing import LabelEncoder
for col in ["Sex","Embarked","Title","Deck"]:
    le = LabelEncoder()
    clean_test[col] = le.fit_transform(clean_test[col])
clean_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,Alone,AgeGroup,FareGroup
0,892,3,1,34.5,0,0,7.8292,1,2,7,1,1,2,0.0
1,893,3,0,47.0,1,0,7.0000,2,3,7,2,0,3,0.0
2,894,2,1,62.0,0,0,9.6875,1,2,7,1,1,4,0.0
3,895,3,1,27.0,0,0,8.6625,2,2,7,1,1,2,0.0
4,896,3,0,22.0,1,1,12.2875,2,3,7,3,0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,27.0,0,0,8.0500,2,2,7,1,1,2,0.0
414,1306,1,0,39.0,0,0,108.9000,0,4,2,1,1,2,0.0
415,1307,3,1,38.5,0,0,7.2500,2,2,7,1,1,2,0.0
416,1308,3,1,27.0,0,0,8.0500,2,2,7,1,1,2,0.0


In [9]:
##making the train test split 
from sklearn.model_selection import train_test_split
target=clean["Survived"]
info=clean.drop(["Survived"],axis=1)

info_train, info_test, target_train, target_test = train_test_split(info, target, test_size=0.2, random_state=42, stratify=target)


In [10]:
#Training the randomforestmodel

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42
)
model.fit(info_train,target_train)

print("Train Accuracy:", model.score(info_train, target_train))
print("Validation Accuracy:", model.score(info_test, target_test))



Train Accuracy: 0.8707865168539326
Validation Accuracy: 0.8379888268156425


In [11]:


#handling test missing  values
# Median fare per class (from train)
fare_by_class = clean.groupby("Pclass")["Fare"].median()

# Mode fare group per class (from train)
faregroup_by_class = clean.groupby("Pclass")["FareGroup"].apply(lambda x: x.mode()[0])

# Replace missing fare and faregroup in test
for idx in clean_test[clean_test["Fare"].isna()].index:
    pclass = clean_test.loc[idx, "Pclass"]
    clean_test.loc[idx, "Fare"] = fare_by_class.loc[pclass]
    clean_test.loc[idx, "FareGroup"] = faregroup_by_class.loc[pclass]
clean_test["AgeGroup"]=pd.cut(clean_test["Age"],bins=[0,12,20,40,60,80],labels=False)
clean_test["FareGroup"]=pd.cut(clean_test["Fare"],4,labels=False)

clean_test.isna().sum()


PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
Deck           0
FamilySize     0
Alone          0
AgeGroup       0
FareGroup      0
dtype: int64

In [12]:
##making the predictions
predictions = model.predict(clean_test)
output = pd.DataFrame({'PassengerId': clean_test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)